In [7]:
import glob
import pandas as pd

from unseen import fileio
from unseen import spatial_selection
from unseen import general_utils

In [8]:
infiles_1990s = sorted(glob.glob('/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-199[5,6,7,8,9]*/atmos_isobaric_daily.zarr.zip'))

In [9]:
infiles_2000s = sorted(glob.glob('/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2*/atmos_isobaric_daily.zarr.zip'))

In [10]:
def is_jja(month):
    return (month >= 6) & (month <= 8)

In [16]:
def subset_days(infiles, threshold, mode):
    """Select a subset of summer days based on a temperature threshold"""
    
    assert mode in ['greater than', 'less than']

    tasmax_df = pd.DataFrame([])
    for infile in infiles:
        print(infile)
        ds = fileio.open_dataset(
            infile,
            variables=['tasmax'],
            metadata_file='/home/599/dbi599/unseen/config/dataset_cafe_daily.yml',
        )
        da_tasmax = spatial_selection.select_point_region(ds['tasmax'], [47.45, 237.69])
        da_tasmax = general_utils.convert_units(da_tasmax, 'C')
        da_tasmax_jja = da_tasmax.sel(time=is_jja(ds['time.month']))

        df_tasmax_jja = da_tasmax_jja.to_dataframe()
        if mode == 'greater than':
            selection = df_tasmax_jja['tasmax'] > threshold
        else:
            selection = df_tasmax_jja['tasmax'] < threshold
        df_tasmax_jja = df_tasmax_jja[selection]
        df_tasmax_jja['infile'] = [infile] * len(df_tasmax_jja)

        tasmax_df = tasmax_df.append(df_tasmax_jja)

    return tasmax_df

## Hottest days

In [23]:
hottest_df = subset_days(infiles_1990s + infiles_2000s, 43, 'greater than')

/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19950501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19951101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19960501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19961101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19970501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19971101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19980501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19981101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19990501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19991101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-20000501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-20001101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-20010501/atmos_isobaric_d

In [24]:
hottest_df.sort_values(by=['tasmax'], ascending=False)

,,tasmax,infile
time,ensemble,,
2001-07-29 12:00:00,62,46.097260,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2000050...
2018-07-22 12:00:00,81,46.044830,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2011050...
2021-08-08 12:00:00,43,45.923462,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2018110...
2020-07-23 12:00:00,56,45.922943,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2017110...
2012-07-09 12:00:00,64,45.889343,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2007050...
...,...,...,...
2008-07-23 12:00:00,7,43.005402,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2001110...
2002-08-13 12:00:00,63,43.004211,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2001050...
2006-07-07 12:00:00,26,43.003387,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-1998050...


## Coolest days

In [21]:
coldest_df = subset_days(infiles_1990s + infiles_2000s, 7, 'less than')

/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19950501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19951101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19960501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19961101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19970501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19971101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19980501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19981101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19990501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-19991101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-20000501/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-20001101/atmos_isobaric_daily.zarr.zip
/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-20010501/atmos_isobaric_d

In [22]:
coldest_df.sort_values(by=['tasmax'], ascending=True)

,,tasmax,infile
time,ensemble,,
2016-06-18 12:00:00,61,4.731201,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2009050...
2004-06-01 12:00:00,2,4.769470,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-1995050...
2010-06-07 12:00:00,6,4.848267,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2008110...
2004-06-01 12:00:00,10,4.975952,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-1995050...
2005-06-03 12:00:00,45,4.988281,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2000050...
...,...,...,...
2021-06-02 12:00:00,85,6.995667,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2015110...
2014-06-05 12:00:00,21,6.995850,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2004110...
2017-06-03 12:00:00,32,6.998138,/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-2009050...
